In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import re 
from scipy import sparse
import time
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_colwidth=300
pd.options.display.max_columns = 100

from sklearn.linear_model import Ridge


# Loading Data 

In [3]:
df = pd.read_csv('train_data_version1.csv')
df.head()

,text,y
0,oh why dear johnny boy part one again do you want a fucking fight what the shit is your problem ooh he s personally attacking me grow up,2.08
1,savant13 is a fat shite,3.58
2,i am english i am i do not want to get blocked again fu k,0.80
3,and why the fuck should i care g e t r i d o f a l l i d i d p l e a s e j a ck a s s,4.58
4,yo dear sir james paul you a fruity ass nigga get a life man unless your happy with editing wikipedia as your profession feel bad for you its probably hard livin life without any friends,2.08


<h3>Text Cleaning</h3>

In [4]:
def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text

# TF-IDF

In [5]:
df = df.dropna(axis = 0)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47991 entries, 0 to 47990
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    47991 non-null  object 
 1   y       47991 non-null  float64
dtypes: float64(1), object(1)
memory usage: 750.0+ KB


In [7]:
vec = TfidfVectorizer(min_df= 3, max_df=0.5, analyzer = 'char_wb', ngram_range = (3,5), max_features = 46000)
vec.fit(df['text'])

TfidfVectorizer(analyzer='char_wb', max_df=0.5, max_features=46000, min_df=3,
                ngram_range=(3, 5))

In [8]:
from gensim.models import KeyedVectors, FastText

fmodel = FastText.load('./FastText-jigsaw-256D/Jigsaw-Fasttext-Word-Embeddings-256D.bin')

# Preparing Data For training 

We used 46000 Features of TFIDF Word Representations, 256 Dimensional FastText Word Embeddings (Trained on Toxic Comments) and then concatenate them to make final dataset for Training a model 

In [9]:
from scipy.sparse import hstack

def splitter(text):
    tokens = []
    
    for word in text.split(' '):
        tokens.append(word)
    
    return tokens

def vectorizer(text):
    tokens = splitter(text)
    
    x1 = vec.transform([text]).toarray()
    x2 = np.mean(fmodel.wv[tokens], axis = 0).reshape(1, -1)
    x = np.concatenate([x1, x2], axis = -1).astype(np.float16)
    del x1
    del x2 
    
    return x   

In below code cell we are creating data which we will feed in model

In [10]:
X_list = []

for text in df.text:
    X_list.append(vectorizer(text))

In [11]:
EMB_DIM = len(vec.vocabulary_) + 256

In [12]:
X_np = np.array(X_list).reshape(-1, EMB_DIM)

In [13]:
from scipy import sparse

X = sparse.csr_matrix(X_np)
del X_np

In [14]:
X

<47991x46256 sparse matrix of type '<class 'numpy.float32'>'
	with 29708370 stored elements in Compressed Sparse Row format>

<h1>Fit Ridge</h1>

In [15]:
%%time
model = Ridge(alpha=0.5)
model.fit(X, df['y'])

CPU times: total: 16min 30s
Wall time: 2min 4s


Ridge(alpha=0.5)

In [16]:
%%time
l_model = Ridge(alpha=1.)
l_model.fit(X, df['y'])

CPU times: total: 12min 51s
Wall time: 1min 36s


Ridge()

In [17]:
%%time
d_model = Ridge(alpha=1.5)
d_model.fit(X, df['y'])

CPU times: total: 11min 57s
Wall time: 1min 30s


Ridge(alpha=1.5)

In [18]:
%%time
s_model = Ridge(alpha=2.)
s_model.fit(X, df['y'])

CPU times: total: 10min 37s
Wall time: 1min 20s


Ridge(alpha=2.0)

## Prediction

In [28]:
p3 = s_model.predict(X)

In [29]:
p3

array([1.5627846 , 1.9564546 , 0.5945374 , ..., 2.6738548 , 1.4661351 ,
       0.40538475], dtype=float32)

In [30]:
df_sub['score'] = p3

In [31]:
df_sub['score'].count()

47991

In [32]:
df_sub['score'].nunique()

43059